In [ ]:
#pip install scikit-surprise
#https://towardsdatascience.com/machine-learning-for-building-recommender-system-in-python-9e4922dd7e97


In [11]:
#pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Clamfighter\Documents\GitHub\Recommendation-System\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
import numpy as np
import pandas as pd

In [2]:
def readFile(file_path, rows=100000):
    data_dict = {'Cust_Id' : [], 'Movie_Id' : [], 'Rating' : [], 'Date' : []}
    f = open(file_path, "r")
    count = 0
    for line in f:
        count += 1
        if count > rows:
            break
            
        if ':' in line:
            movidId = line[:-2] # remove the last character ':'
            movieId = int(movidId)
        else:
            customerID, rating, date = line.split(',')
            data_dict['Cust_Id'].append(customerID)
            data_dict['Movie_Id'].append(movieId)
            data_dict['Rating'].append(rating)
            data_dict['Date'].append(date.rstrip("\n"))
    f.close()
            
    return pd.DataFrame(data_dict)

In [9]:
df1 = readFile('F:/Recommendation-System/archive/combined_data_1.txt', rows=100000)
df2 = readFile('F:/Recommendation-System/archive/combined_data_2.txt', rows=100000)
df3 = readFile('F:/Recommendation-System/archive/combined_data_3.txt', rows=100000)
df4 = readFile('F:/Recommendation-System/archive/combined_data_4.txt', rows=100000)
df1['Rating'] = df1['Rating'].astype(float)
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

In [13]:
df = df1.copy()
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df.index = np.arange(0,len(df))
df.head(10)

C:\Users\Clamfighter\AppData\Local\Temp\ipykernel_9236\2231859869.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\Clamfighter\AppData\Local\Temp\ipykernel_9236\2231859869.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df3)
C:\Users\Clamfighter\AppData\Local\Temp\ipykernel_9236\2231859869.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df4)


,Cust_Id,Movie_Id,Rating,Date
0,1488844,1,3.0,2005-09-06
1,822109,1,5.0,2005-05-13
2,885013,1,4.0,2005-10-19
3,30878,1,4.0,2005-12-26
4,823519,1,3.0,2004-05-03
5,893988,1,3.0,2005-11-17
6,124105,1,4.0,2004-08-05
7,1248029,1,3.0,2004-04-22
8,1842128,1,4.0,2004-05-09
9,2238063,1,3.0,2005-05-11


In [20]:
df_title = pd.read_csv('F:/Recommendation-System/archive/movie_titles.csv', encoding = "ISO-8859-1", header = None,names = ['Movie_Id', 'Year', 'Name'])
df_title.head(10)

,Movie_Id,Year,Name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [21]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
reader = Reader()
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)
svd = SVD()

In [22]:
# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0154  1.0141  1.0183  1.0151  1.0118  1.0149  0.0021  
MAE (testset)     0.8102  0.8090  0.8132  0.8111  0.8039  0.8095  0.0031  
Fit time          25.53   21.96   22.35   22.40   22.03   22.85   1.35    
Test time         0.80    0.58    0.59    0.56    0.61    0.63    0.09    


{'test_rmse': array([1.01537182, 1.01407252, 1.01825619, 1.01510714, 1.01177738]),
 'test_mae': array([0.81016633, 0.80902919, 0.81318549, 0.81114323, 0.80388121]),
 'fit_time': (25.528687715530396,
  21.960041761398315,
  22.348022937774658,
  22.39787793159485,
  22.02566909790039),
 'test_time': (0.7986714839935303,
  0.5799684524536133,
  0.5893592834472656,
  0.5599491596221924,
  0.6148843765258789)}

In [23]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [24]:
titles = df_title.copy()
titles['Estimate_Score'] = titles['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

In [25]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head(10)

,Movie_Id,Year,Name,Estimate_Score
12,13,2003.0,Lord of the Rings: The Return of the King: Ext...,4.502260
9235,9236,1998.0,South Park: Season 2,4.092141
4505,4506,1961.0,Breakfast at Tiffany's,4.030432
24,25,1997.0,Inspector Morse 31: Death Is Now My Neighbour,4.017736
13379,13380,1949.0,Stray Dog,3.993426
13374,13375,1963.0,Andy Griffith Show: Classic Favorites,3.979081
4508,4509,1977.0,Little House on the Prairie: Season 4,3.970040
4,5,2004.0,The Rise and Fall of ECW,3.955028
4520,4521,2002.0,Wire in the Blood: Justice Painted Blind,3.946480
13377,13378,1940.0,His Girl Friday,3.922960
